In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import Sequential,MaxPool2d,Linear,ReLU
from functions_pytorch import *
from pytorch_util import *
from pytorch_models import ConvBatchRelu
from albumentations_helper import create_transform
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
import pickle
import numpy as np
import pretrainedmodels
from functools import partial

In [2]:
## configs ##
color = True
#shapes = (3,224,224)
HalfBatch = 8
outDim = 500
distanceFun = l2_distance
distanceFun_np = l2_distance_np
TTASize = 4
pct = 0.1

In [56]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [57]:
# combine whales with one image with new whale
newWhale_train2 = newWhale_train + [i[0] for i in Ids_train.loc[Ids_train.length2 == 1].Imgs.tolist()]
Ids_train2 = Ids_train.loc[Ids_train.length2>1]

Set up dataloader

In [38]:
aug = Compose([RandomContrast(p=0.5),RandomBrightness(p=0.5),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=25,scale_limit=0.05,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

In [39]:
gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch,False,num_workers=3,drop_last=True)

Set up training

In [6]:
# convNet = Sequential(ConvBatchRelu(3,8,5,stride=2),\
#                      ConvBatchRelu(8,16,5,stride=2),\
#                      ConvBatchRelu(16,32,5,stride=2),\
#                      MaxPool2d(2))

# fcNet = Sequential(Linear(4608,32),ReLU(True),Linear(32,6))

# with torch.no_grad():
#     fcNet[2].weight.data.zero_()
#     fcNet[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

# stn = SpatialTransformerNet(convNet,fcNet)

In [7]:
#base = pretrainedmodels.resnet50()
# base = pretrainedmodels.densenet121()
# base = fine_tune_pretrainedmodels(base, outDim)
# model = Sequential(stn,base).to('cuda:0')

In [40]:
model = pretrainedmodels.densenet121()
model = fine_tune_pretrainedmodels(model, outDim).to('cuda:0')

In [41]:
loss_func = loss_func_generator_softmax(HalfBatch,distanceFun)

In [42]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [43]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:1.3807016319590188, val_loss:0.7200477719306946
epoch:1, train_loss:1.090433942652791, val_loss:0.6072275638580322
epoch:2, train_loss:0.9916346464815036, val_loss:0.53001868724823
epoch:3, train_loss:0.9396744084244217, val_loss:0.5114064812660217
epoch:4, train_loss:0.9167857646453575, val_loss:0.5083933472633362
Training completed in 99.3218138217926s


Fine-tune previous layers

In [ ]:
# for ResNet50
# set_requires_grad([model[1].layer4,model[1].layer3,model[1].layer2],True)
# opt = Adam([
#             {"params": model[1].layer4.parameters(), "lr": 1e-5},
#             {"params": model[1].layer3.parameters(), "lr": 1e-6},
#             {"params": model[1].layer2.parameters(), "lr": 5e-7},
#             {"params": model[1].last_linear.parameters(), "lr": 2e-5},
#             {"params": model[0].parameters(), "lr": 2e-5}
#             ])

In [44]:
# for DenseNet121
name_list = ['denseblock3','transition3','denseblock4','norm5']
set_requires_grad_paraList(gather_parameter_nameList(name_list,model),True)

In [45]:
opt = Adam([
            {"params": gather_parameter_nameList(['denseblock4','norm5'],model), "lr": 1e-4},
            {"params": gather_parameter_nameList(['denseblock3','transition3'],model), "lr": 1e-5},
            {"params": gather_parameter_nameList(['denseblock2','transition2'],model), "lr": 1e-6},
            {"params": model.last_linear.parameters(), "lr": 2e-4}
            ])

In [46]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.7278579282548909, val_loss:0.33879828453063965
epoch:1, train_loss:0.5525988931335061, val_loss:0.3007964491844177
epoch:2, train_loss:0.472943750540904, val_loss:0.24787844717502594
epoch:3, train_loss:0.40091214386003265, val_loss:0.2792761027812958
epoch:4, train_loss:0.3689443282986599, val_loss:0.23102129995822906
Training completed in 161.5082721710205s


In [48]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk1_softmax')

In [ ]:
# MAP -> 0.24035250463821894

HalfBatch*2

In [65]:
gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch*2,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch*2,False,num_workers=3,drop_last=True)

In [66]:
loss_func = loss_func_generator_softmax(HalfBatch*2,distanceFun)

In [67]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.5718846795011739, val_loss:0.3294067978858948
epoch:1, train_loss:0.5004904678936213, val_loss:0.30512407422065735
epoch:2, train_loss:0.46436618071029095, val_loss:0.2928871214389801
epoch:3, train_loss:0.4369352531156253, val_loss:0.27999189496040344
epoch:4, train_loss:0.40400626376026966, val_loss:0.27369144558906555
Training completed in 121.09027218818665s


In [76]:
model = fit(3, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.3986337505525253, val_loss:0.24848712980747223
epoch:1, train_loss:0.35550299628836207, val_loss:0.2301209717988968
epoch:2, train_loss:0.34285374909965066, val_loss:0.23225192725658417
Training completed in 72.52986192703247s


In [77]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk2_softmax')

In [ ]:
# MAP -> 0.3799628942486086

HalfBatch*3

In [85]:
gen_train = TripletGenerator(Ids_train2,newWhale_train2,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch*3,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch*3,False,num_workers=3,drop_last=True)

In [86]:
loss_func = loss_func_generator_softmax(HalfBatch*3,distanceFun)

In [87]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.4345991045969432, val_loss:0.3216629922389984
epoch:1, train_loss:0.4196563957289594, val_loss:0.31008440256118774
epoch:2, train_loss:0.3710389743391119, val_loss:0.2892552614212036
epoch:3, train_loss:0.35855810710641206, val_loss:0.292125403881073
epoch:4, train_loss:0.3675323299086485, val_loss:0.2807040512561798
Training completed in 110.31430578231812s


In [95]:
model = fit(3, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.34320533929056807, val_loss:0.30140191316604614
epoch:1, train_loss:0.33124642092429224, val_loss:0.31471681594848633
epoch:2, train_loss:0.31387786243538385, val_loss:0.31081876158714294
Training completed in 66.18699336051941s


In [104]:
model = fit(3, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.3175262591694711, val_loss:0.3300155699253082
epoch:1, train_loss:0.2952021814760615, val_loss:0.31774693727493286
epoch:2, train_loss:0.2715662341625964, val_loss:0.29719290137290955
Training completed in 66.56434273719788s


In [103]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk3_softmax')

In [ ]:
# MAP -> 0.487012987012987

Validation

In [105]:
pred_train_gen = PredictGenerator(Ids_train.Imgs.tolist(),transform_test,TTASize, True)
pred_val_gen = PredictGenerator(Ids_val.Imgs.tolist(),transform_test,TTASize, True)

In [106]:
feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

In [107]:
feature_train = feature_train.reshape(len(pred_train_gen),TTASize,outDim)
feature_val = feature_val.reshape(len(pred_val_gen),TTASize,outDim)

In [108]:
aggFun = partial(np.quantile,q=pct,axis=(1,2))

In [109]:
predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

In [110]:
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)

In [111]:
MAP(labels,predicts)

0.487012987012987